In [1]:
import pandas as pd
import utils
pd.options.display.max_rows = 100000
pd.set_option('display.max_colwidth', -1)

PATH="C:/Users/bdolicki/Documents/Git/multilingual-analysis/code/analysing_predictions/panx"

C:\Users\bdolicki\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [11]:
# we use a sample of 3 languages as an example, but you can use any language that is included in all_langs
all_langs = ['af', 'ar','bn', 'de', 'el', 'en', 'es', 'et', 'eu', 'fa', 'fi', 'fr', 'he', 'hi', 'hu', 'id', 'it', 'ja', 'ka', 'kk', 'ko', 'ml', 'mr', 'ms', 'my', 'nl', 'pt', 'ru', 'sw', 'ta', 'te', 'th', 'tl', 'ur', 'vi', 'zh']
train_langs = ['af', 'ar', 'en']
test_langs = ['af', 'ar', 'en']

In [12]:
from collections import defaultdict

all_examples = defaultdict(dict)

In [13]:
# show most frequently confused POS tags for a language pair, conf is the confusion matrix
def most_confused_pos(conf):
    conf_count = pd.DataFrame()
    conf_count["count"] = conf.sum().sort_values(ascending=False) 
    conf_count["percent"] = conf_count/conf_count.sum()
    print("Most confused POS tags")
    display(conf_count[:5])
    return conf_count
    
# show most frequently confused POS pairs for a language pair, conf is the confusion matrix
def most_confused_pos_pairs(conf):
    sparse_conf_df = conf.reset_index().melt(id_vars="index",var_name="actual",value_name="count").rename(columns={"index":"pred"})
    sparse_conf_df["percent"] = sparse_conf_df["count"]/sparse_conf_df["count"].sum()
    sparse_conf_df = sparse_conf_df.sort_values(by="count",ascending=False)
    print("Most confused POS pairs")
    display(sparse_conf_df.head())

# show top 5 most frequently confused words and how many times each word was assigned a particular POS tag
# useful to see if a word has multiple POS tags or not
# mistakes is the df with mistaken words
def most_mistaken_word_with_tags(mistakes):
    for token in mistakes_df.token.value_counts()[:5].index:
        token_tags = defaultdict(int)
        for i,sentence in enumerate(pa.all_sentences):
            words = sentence.split()
            for j,word in enumerate(words):
                if word==token:
                    tag = pa.all_tags[i].split()[j]
                    token_tags[tag] += 1
        print("Possible tags for the word",token)
        print(dict(token_tags))
        
# show how many times a word was mistaken together with its frequency
def mistaken_words_with_word_freq(pa):
    def is_all(actual, pred=None):
        return True
    all_df, _ = pa.transform_data(is_all,conf_matrix=True)
    relative = pd.DataFrame()
    relative["% mistaken"] = mistakes_df.token.value_counts().divide(all_df.token.value_counts()).sort_values(ascending=False)
    relative["all_count"] = all_df.token.value_counts()
    relative["mistaken_count"] = mistakes_df.token.value_counts()
    relative = relative.dropna()
    print("Relative mistaken word frequency")
    display(relative.sort_values(by="% mistaken",ascending=False).head())

#   show most mistaken words per target language from all source languages
def most_mistaken_words_per_target_func(most_mistaken_words_list):
    print("Most mistaken words per target language from all source languages")
    display(pd.Series(most_mistaken_words_per_target).value_counts()[:5])

#   show most mistaken POS tags per target language from all source languages
def most_mistaken_pos_per_target_func(most_mistaken_pos_list):
    print("Most mistaken POS tags per target language from all source languages")
    display(pd.Series(most_mistaken_pos_per_target).value_counts()[:5])

#   show top 1 mistaken POS tag from all target languages
def top1_mistaken_pos_per_target(most_mistaken_pos_lang):
    print("Top 1 mistaken POS tag for all target languages")
    display(pd.DataFrame.from_dict(most_mistaken_pos_lang))

In [14]:
most_mistaken_pos_lang = {"lang":[],"tag":[]}
mistakes = defaultdict(dict)

for test_lang in test_langs:
#     aggregate most mistaken words/pos tags per target language for all source languages
    most_mistaken_words_per_target = []
    most_mistaken_pos_per_target = []
    print("test",test_lang)
    for train_lang in train_langs:
        print('train',train_lang)
        pa = utils.PredictionsAnalyzer(task="ner")
        ud_pos = pa.read_tag_data(f"{PATH}/dev_sets/dev-{test_lang}.tsv",f"{PATH}/results/dev_{train_lang}_{test_lang}_predictions.txt")
        def is_pred_wrong(actual,pred=None):
            return actual!=pred

        mistakes_df, conf_df = pa.transform_data(is_pred_wrong,conf_matrix=True)
#         store df with mistakes for all language pairs if you want to perform further analysis
#         when running for a big subset of languages it might cause memory issues
        mistakes[train_lang][test_lang] = mistakes_df
        
        conf_count = most_confused_pos(conf_df)
        most_confused_pos_pairs(conf_df)
        most_mistaken_word_with_tags(mistakes_df)
        
        if train_lang==test_lang:
            print("Supervised")
        else:
            print("Zero-shot")

        most_mistaken_words_per_target += list(mistakes_df.token.value_counts()[:2].index)
        most_mistaken_pos_per_target += [conf_count.index[0]]
        
        mistaken_words_with_word_freq(pa)
        
    most_mistaken_words_per_target_func(most_mistaken_words_per_target)
    most_mistaken_pos_per_target_func(most_mistaken_pos_per_target)
    
    most_mistaken_pos_lang["lang"].append(test_lang)
    most_mistaken_pos_lang["tag"].append(pd.Series(most_mistaken_pos_per_target).value_counts().index[0])


test af
train af
Most confused POS tags


,count,percent
O,71.0,0.300847
B-ORG,55.0,0.233051
I-ORG,37.0,0.156780
B-LOC,31.0,0.131356
I-LOC,21.0,0.088983


Most confused POS pairs


,pred,actual,count,percent
7,B-LOC,B-ORG,29.0,0.122881
42,B-LOC,O,23.0,0.097458
46,I-ORG,O,21.0,0.088983
1,B-ORG,B-LOC,19.0,0.080508
31,I-LOC,I-ORG,17.0,0.072034


Possible tags for the word van
{'O': 251, 'I-ORG': 59, 'I-LOC': 20, 'I-PER': 38}
Possible tags for the word die
{'O': 551, 'I-ORG': 14, 'I-PER': 6, 'I-LOC': 4}
Possible tags for the word ,
{'O': 460, 'I-PER': 4, 'I-LOC': 10, 'I-ORG': 3}
Possible tags for the word (
{'O': 133, 'I-ORG': 17, 'I-LOC': 6}
Possible tags for the word .
{'O': 520, 'I-PER': 4, 'I-ORG': 4, 'I-LOC': 1}
Supervised
Relative mistaken word frequency


...

,% mistaken,all_count,mistaken_count
피하는,1.0,1,1.0
Wheeler,1.0,1,1.0
Surveyor,1.0,1,1.0
ramidus,1.0,1,1.0
Swinnerton,1.0,1,1.0


Most mistaken words per target language from all source languages


,     3
of    3
dtype: int64

Most mistaken POS tags per target language from all source languages


I-ORG    3
dtype: int64

In [15]:
top1_mistaken_pos_per_target(most_mistaken_pos_lang)

Top 1 mistaken POS tag for all target languages


,lang,tag
0,af,O
1,ar,I-LOC
2,en,I-ORG


## Further analysis
We can use stored tables to zoom in on particular language pairs. Here we look at the predictions when Afrikaan was the source language and English was the target language.

In [19]:
mistakes_df = mistakes['af']['en']
# show examples when I-ORG was the actual tag
mistakes_df[mistakes_df['actual']=='I-ORG'].head()

,predicted,actual,token,sentence,actual_tokens
6,O,I-ORG,Red,Fairground]] '' by Simply Red '' Bad Romance '' by Lady Gaga,O O O B-ORG I-ORG O B-ORG I-ORG O O B-PER I-PER
8,O,I-ORG,Romance,Fairground]] '' by Simply Red '' Bad Romance '' by Lady Gaga,O O O B-ORG I-ORG O B-ORG I-ORG O O B-PER I-PER
17,O,I-ORG,(,Parodius ( series ),B-ORG I-ORG I-ORG I-ORG
18,O,I-ORG,series,Parodius ( series ),B-ORG I-ORG I-ORG I-ORG
19,O,I-ORG,),Parodius ( series ),B-ORG I-ORG I-ORG I-ORG
